## Code to generate puzzles

In [4]:
import numpy as np
import pickle
import random
import networkx as nx
import matplotlib.pyplot as plt
import os
import json

Function to sample a Gaussian n in the range [1, M]

In [5]:
# Function to sample a Gaussian n in the range [1, M]
M = 10
def sample_gaussian_n(mean=M/2, std_dev=2, min_n=1, max_n=M):
    n = int(np.clip(np.random.normal(mean, std_dev), min_n, max_n))
    return n

Function to load the flat dataset

In [6]:
# Function to load the flat dataset
def load_flat_dataset(file_path='data/NIS/NISdb_flat.pkl'):
    with open(file_path, 'rb') as f:
        flat_dataset = pickle.load(f)
    return flat_dataset

Function to Sample a random string of size n from NISdb_flat

In [7]:
# Function to Sample a random string of size n from NISdb_flat
def sample_random_string(n, file_path='data/NIS/NISdb_flat.pkl'):
    flat_dataset = load_flat_dataset(file_path)
    
    if n not in flat_dataset or not flat_dataset[n]:
        raise ValueError(f"No strings of length {n} found in the dataset.")
    
    return random.choice(flat_dataset[n])

Function to create transitions

Sample two numbers (exponental) between 1 to min(2n,M) p and q, sample strings of size p and q from NISdb_flat

In [8]:
# Function to create transitions
def create_transition(n, file_path='data/NIS/NISdb_flat.pkl'):
    Max_size = min(2*n, M)
    while True:
        p = int(np.clip(np.random.exponential(scale=n), 1, Max_size))
        q = int(np.clip(np.random.exponential(scale=n), 1, Max_size))
        
        s1 = sample_random_string(p, file_path)
        s2 = sample_random_string(q, file_path)
        
        if s1 != s2:
            break
        
    
    return [s1, s2]

Function to create an array of transitions of size t for a given n

In [9]:
# Transitions are arrays of size 2 of strings
# Function to create an array of transitions of size t for a given n
def create_transitions_array(n, t, file_path='data/NIS/NISdb_flat.pkl'):
    transitions = []
    for _ in range(t):
        transitions.append(create_transition(n, file_path))
    return transitions

Function to apply a transition [s1, s2] to a string s

In [10]:
# To apply a transition [s1, s2] to a string s, we find the first occurrence of s1 in s and replace it with s2:

# Function to apply a transition [s1, s2] to a string s
def apply_transition(s, transition):
    s1, s2 = transition
    if s1 not in s:
        return None
    return s.replace(s1, s2, 1)

Graph generator
1. Sample n
2. Create t transitions (t =5) 
3. Create a graph with root node as a string of size n and t transitions upto depth d

Graph generator function

In [11]:
# Graph generator function
def generate_graph(n=None, t=3, d=3, file_path='data/NIS/NISdb_flat.pkl'):
    if n is None:
        n = sample_gaussian_n()
    root = sample_random_string(n, file_path)
    # print(f"Random string of length {n}: {root}")
    transitions = create_transitions_array(n, t, file_path)
    # print(f"Transitions array: {transitions}")
    
    G = nx.DiGraph()
    G.add_node((-1, -1, -1), string=root)
    current_level = [((-1, -1, -1), root)]
    
    for depth in range(d):
        next_level = []
        for node, node_string in current_level:
            for i, transition in enumerate(transitions):
                new_node_string = apply_transition(node_string, transition)
                if new_node_string is not None:
                    new_node = tuple(list(node[:depth]) + [i] + [-1] * (2 - depth))
                    G.add_node(new_node, string=new_node_string)
                    G.add_edge(node, new_node, label=f"{transition[0]} -> {transition[1]}")
                    next_level.append((new_node, new_node_string))
        current_level = next_level
    
    return G

In [12]:
# Function to plot the graph
def plot_graph(G):
    pos = nx.shell_layout(G)
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'string'), node_size=3000, node_color="skyblue", font_size=10, font_weight="bold", arrows=True)
    
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    
    plt.show()

In [13]:
# # Example usage
# if __name__ == "__main__":    
#     n = None  
#     graph = generate_graph(n,3,5)
#     print(f"Generated graph: {graph}")
    
#     plot_graph(graph)

Rather than generating a graph, now we shall directly generate a puzzle 

Single Path generator function

In [14]:
# Graph generator function to generate only one path
def generate_single_path_graph(n=sample_gaussian_n(), t=3, d=3, file_path='data/NIS/NISdb_flat.pkl'):
    root = sample_random_string(n, file_path)
    print(f"Random string of length {n}: {root}")
    
    G = nx.DiGraph()
    G.add_node((-1, -1, -1), string=root)
    current_node = (-1, -1, -1)
    current_string = root
    successful_transitions = 0
    attempts = 0
    solution = []
    
    while successful_transitions < d:
        if attempts >= 4 * d:
            # Restart with a new string if too many attempts fail
            root = sample_random_string(n, file_path)
            print(f"Restarting with new random string of length {n}: {root}")
            G = nx.DiGraph()
            G.add_node((-1, -1, -1), string=root)
            current_node = (-1, -1, -1)
            current_string = root
            successful_transitions = 0
            attempts = 0
            solution = []
        
        transitions = create_transitions_array(n, t, file_path)
        transition = random.choice(transitions)
        new_string = apply_transition(current_string, transition)
        
        if new_string is not None:
            new_node = tuple(list(current_node[:successful_transitions]) + [transitions.index(transition)] + [-1] * (2 - successful_transitions))
            G.add_node(new_node, string=new_string)
            G.add_edge(current_node, new_node, label=f"{transition[0]} -> {transition[1]}")
            current_node = new_node
            current_string = new_string
            successful_transitions += 1
            solution.append(transitions.index(transition))
        attempts += 1
    
    return G, solution

In [15]:
# # Example usage
# if __name__ == "__main__":
#     graph = generate_single_path_graph()
#     print(f"Generated graph: {graph}")
    
#     plot_graph(graph)

In [16]:
# Function to count leaf nodes in a graph
def count_leaf_nodes(G):
    leaf_nodes = [node for node in G.nodes if G.out_degree(node) == 0]
    return len(leaf_nodes)

# Function to plot the frequency chart of leaf counts
def plot_leaf_count_frequency(leaf_counts):
    plt.figure(figsize=(10, 6))
    plt.hist(leaf_counts, bins=range(min(leaf_counts), max(leaf_counts) + 1), edgecolor='black')
    plt.title(f'Frequency of Leaf Counts in {len(leaf_counts)} Graphs')
    plt.xlabel('Number of Leaf Nodes')
    plt.ylabel('Frequency')
    plt.show()


# Function to generate <count> graphs, save them to files, and return the file names and leaf counts
def generate_graphs(count=100):
    file_names = []
    leaf_counts = []
    
    for i in range(count):
        G = generate_graph()
        leaf_count = count_leaf_nodes(G)
        leaf_counts.append(leaf_count)
        
        file_name = f'Graphs_{count}_{i}.pkl'
        with open(file_name, 'wb') as f:
            pickle.dump(G, f)
        
        file_names.append(file_name)
        # print(f"Graph {i+1}: {leaf_count} leaf nodes, saved to {file_name}")
    
    return file_names, leaf_counts

In [17]:
# # Example usage
# if __name__ == "__main__":
#     file_names, leaf_counts = generate_graphs()
    
#     # Calculate and print the average number of leaf nodes
#     average_leaf_count = sum(leaf_counts) / len(leaf_counts)
#     print(f"Average number of leaf nodes: {average_leaf_count}")
    
#     # Plot the frequency chart of leaf counts
#     plot_leaf_count_frequency(leaf_counts)
    
#     # # Plot the first graph as an example
#     # with open(file_names[0], 'rb') as f:
#     #     first_graph = pickle.load(f)
#     # plot_graph(first_graph)

Should we keep t variable or fixed for a dataset

In [20]:
# Function to generate puzzle instances

def generate_single_puzzles_and_solutions(t=3, d=3, count=10):
    puzzles = []
    solutions = []
    
    for i in range(count):
        G, solution = generate_single_path_graph(t=t, d=d)
        puzzles.append(G)
        solutions.append(solution)
    
    return puzzles, solutions

issues
1. directory name and files structure not working as intended
2. More importantly, Puzzzle solutions are wrong: plot graph of puzzle and check


Problem: Most solutions will have transition order 01234, must mix later